In [ ]:
# Write your imports here

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import zipfile
from io import TextIOWrapper
import re

# Data Visualization and Exploratory Data Analysis Lab
## Visualizing and exploring data. The data mining process

In this lab, you'll get acquainted with the most streamed songs on Spotify in 2024. The dataset and its associated metadata can be found [here](https://www.kaggle.com/datasets/nelgiriyewithana/most-streamed-spotify-songs-2024). The version you'll need is provided in the `data/` folder.

You know the drill. Do what you can / want / need to answer the questions to the best of your ability. Answers do not need to be trivial, or even the same among different people.

### Problem 1. Read the dataset (1 point)
Read the file without unzipping it first. You can try a different character encoding, like `unicode_escape`. Don't worry too much about weird characters.

In [ ]:
zip_path = 'data\spotify_most_streamed_2024.zip'

csv_file_name = 'Most Streamed Spotify Songs 2024.csv'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    with zip_ref.open(csv_file_name) as file:
        with TextIOWrapper(file, encoding='unicode_escape') as text_file:
            spotify_df = pd.read_csv(text_file)

In [ ]:
spotify_df

### Problem 2. Perform some cleaning (1 point)
Ensure all data has been read correctly; check the data types. Give the columns better names (e.g. `all_time_rank`, `track_score`, etc.). To do so, try to use `apply()` instead of a manual mapping between old and new name. Get rid of any unnecessary ones.

In [ ]:
spotify_df.dtypes

First we will take care of the column names and then we will proceed to the cleaning - the columns with reach, count, streams, etc all need to be floats or integers

In [ ]:
def pythonic_columns(column_name):
    column_name = column_name.lower()
    # Replace spaces with underscores
    column_name = column_name.replace(' ', '_')
    # Replace special characters with underscores
    column_name = re.sub(r'\W+', '_', column_name)
    # Ensure column names are valid Python identifiers
    column_name = re.sub(r'^[^a-zA-Z_]+', '', column_name)
    return column_name

spotify_df.columns = [pythonic_columns(col) for col in spotify_df.columns]

In [ ]:
spotify_df.columns

In [ ]:
spotify_df.dtypes

First we will take care of the release date column, where we will convert all to datetime.

In [ ]:
spotify_df['release_date']

In [ ]:
for col in spotify_df.columns:
        try:
            # Attempt to convert the column to datetime, assuming MM/DD/YYYY format
            spotify_df[col] = pd.to_datetime(spotify_df[col], format='%m/%d/%Y')
        except (ValueError, TypeError):
            # If conversion fails, leave the column as is
            pass

Next, off to the all time rank. It clearly needs to be integer.

In [ ]:
spotify_df['all_time_rank'] = spotify_df['all_time_rank'].str.replace(',', '').astype(int)


Now track scores need to be floats.

In [ ]:
spotify_df['track_score'] = spotify_df['track_score'].astype(float)

deezer_playlist_reach, pandora_streams, pandora_track_stations, soundcloud_streams, shazam_counts, airplay_spins, siriusxm_spins, youtube_views, youtube_likes, tiktok_posts, tiktok_likes, tiktok_views, youtube_playlist_reach, spotify_streams, spotify_playlist_count, spotify_playlist_reach will all undergo the same process in order to be converted to integers.

In [ ]:
columns_to_convert = [
    'deezer_playlist_reach', 'pandora_streams', 
    'pandora_track_stations', 'soundcloud_streams', 'shazam_counts', 'airplay_spins', 'siriusxm_spins', 
    'youtube_views', 'youtube_likes', 'tiktok_posts', 'tiktok_likes', 'tiktok_views', 'youtube_playlist_reach', 
    'spotify_streams', 'spotify_playlist_count', 'spotify_playlist_reach'
]

def convert_column(df, column):
    df[column] = df[column].astype(str).str.replace(',', '')
    df[column] = pd.to_numeric(df[column], errors='coerce').astype('Int64')

for column in columns_to_convert:
    convert_column(spotify_df, column)

In [ ]:
spotify_df.dtypes

Everything looks clear now, we will decide which columns seem redundant once we get started analysing them.

### Problem 3. Most productive artists (1 point)
Who are the five artists with the most songs in the dataset?

Who are the five "clean-mouthed" artists (i.e., with no explicit songs)? **Note:** We're not going into details but we can start a discussion about whether a song needs swearing to be popular.

In [ ]:
artist_song_count = spotify_df['artist'].value_counts().head(5)
artist_song_count

The answer to the first question would be Drake, Taylor Swift, Bad Bunny, KAROL G and The Weeknd.

In [ ]:
clean_artists = spotify_df[spotify_df['explicit_track'] == 0]['artist'].value_counts()

clean_artists = clean_artists[clean_artists.index.isin(spotify_df.groupby('artist')['explicit_track'].sum()[spotify_df.groupby('artist')['explicit_track'].sum() == 0].index)]
clean_artists = clean_artists.head(5)

clean_artists

The five clean-mouth artists with most songs on Spotify are Imagine Dragons, BTS, Linkin Park, Luke Combs and Ozuna.

### Problem 4. Most streamed artists (1 point)
And who are the top five most streamed (as measured by Spotify streams) artists?

In [ ]:
artist_streams = spotify_df.groupby('artist')['spotify_streams'].sum()
top_5_artists_by_streams = artist_streams.sort_values(ascending=False).head(5)

top_5_artists_by_streams

Bad Bunny, The Weeknd, Drake, Taylor Swift, Post Malone hot the top 5 positions for most streams on Spotify.

### Problem 5. Songs by year and month (1 point)
How many songs have been released each year? Present an appropriate plot. Can you explain the behavior of the plot for 2024?

How about months? Is / Are there (a) popular month(s) to release music?

In [ ]:
spotify_df['release_year'] = spotify_df['release_date'].dt.year
songs_per_year = spotify_df['release_year'].value_counts().sort_index()

spotify_df['release_month'] = spotify_df['release_date'].dt.month
songs_per_month = spotify_df['release_month'].value_counts().sort_index()

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(songs_per_year.index, songs_per_year.values, color='skyblue')
plt.title('Number of Songs Released Each Year')
plt.xlabel('Year')
plt.ylabel('Number of Songs')
plt.xticks(rotation=45)
plt.show()

plt.figure(figsize=(12, 6))
plt.bar(songs_per_month.index, songs_per_month.values, color='lightgreen')
plt.title('Number of Songs Released Each Month')
plt.xlabel('Month')
plt.ylabel('Number of Songs')
plt.xticks(rotation=45)
plt.show()

There is a clear upward trend throughout the years, with the released songs peaking in 2023 with nearly 1200 new songs. In 2024 a decrease is observed slightly above 600, half the value from a year earlier, which could be explained by the fact that the year is incomplete, thus the data and the plot would look different by the end of the year. 

At the same time the monthly releases fluctuate with just 2 obvious peaks in January and May which seem like the most favourable months for artists with approximately 500 new tracks. December holds the bottom position being the month with the least new releases, just below 300. 

### Problem 6. Playlists (2 points)
Is there any connection (correlation) between users adding a song to playlists in one service, or another? Only Spotify, Apple, Deezer, and Amazon offer the ability to add a song to a playlist. Find a way to plot all these relationships at the same time, and analyze them. Experiment with different types of correlations.

In [ ]:
playlist_data = spotify_df[['spotify_playlist_count', 'apple_music_playlist_count', 'deezer_playlist_count', 'amazon_playlist_count']]

pearson_corr = playlist_data.corr(method='pearson')
spearman_corr = playlist_data.corr(method='spearman')
kendall_corr = playlist_data.corr(method='kendall')

We are going to dive into three different types of correlation in order to examine the relationship between user's behaviour on different services. 

* Pearson Correlation measures linear relationships.
* Spearman Correlation measures monotonic relationships regardless of their linearity.
* Kendall Correlation measures the strength of dependence between two variables.

With heatmaps we have a visual to help us understand the connections with colour gradient. 

In [ ]:
plt.figure(figsize=(18, 5))

plt.subplot(1, 3, 1)
sns.heatmap(pearson_corr, annot=True, cmap='coolwarm')
plt.title('Pearson Correlation')

plt.subplot(1, 3, 2)
sns.heatmap(spearman_corr, annot=True, cmap='coolwarm')
plt.title('Spearman Correlation')

plt.subplot(1, 3, 3)
sns.heatmap(kendall_corr, annot=True, cmap='coolwarm')
plt.title('Kendall Correlation')

plt.tight_layout()
plt.show()


What we can deduce from the heatmaps is that there is a direct linear correlation between adding songs on apple music playlists and on deezer playlists, at the same time people on spotify are not dependent on amazon and vice versa. Another thing that is clear is that apple music users are more versatile in their choice of music services, and there tends to be a stronger connection between adding songs on apple music and on any other provider. 

### Problem 7. YouTube views and likes (1 point)
What is the relationship between YouTube views and likes? Present an appropriate plot. 

What is the mean YouTube views-to-likes ratio? What is its distribution? Find a way to plot it and describe it.

We will start by calculating the ratio and then take its mean value. With a scatter plot we will represent the ratio between youtube views and likes, and with a histogram we will show the frequency of the mean we've calculated.

In [ ]:
spotify_df['views_to_likes_ratio'] = spotify_df['youtube_views'] / spotify_df['youtube_likes']

mean_ratio = spotify_df['views_to_likes_ratio'].mean()


In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='youtube_views', y='youtube_likes', data=spotify_df)
plt.title('Relationship between YouTube Views and Likes')
plt.xlabel('YouTube Views')
plt.ylabel('YouTube Likes')
plt.show()

The concentration of points indicates that the majority of songs have a relatively low number of views. Moreover, for the songs with low views, there is also a low number of likes, yet it is within a proportional range. The outliers, represented by the plot, indicate both exceptionally popular videos with low engagement rate and videos with high engagement, though not as popular.

Now, we will proceed to plotting a histogram to better understand the relationship between views and likes on youtube.

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(spotify_df['views_to_likes_ratio'], kde=True, bins=100, color='skyblue')
plt.title('Distribution of YouTube Views-to-Likes Ratio')
plt.xlabel('Views-to-Likes Ratio')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

This distribution appears to be quite skewed, and this may call for another way of plotting it. First, we are going to apply a natural logarithm to each data point, which will handle zero values. This will normalise the distribution and will reduce the impact of the outliers seen in the previous two plots. We will also add Kernel Density Estimate (KDE) plot to the histogram, which will provide a smoothed, continuous version of the histogram, which will help us visualise the distribution's shape.

In [ ]:
spotify_df['log_views_to_likes_ratio'] = np.log1p(spotify_df['views_to_likes_ratio'])

plt.figure(figsize=(10, 6))
sns.histplot(spotify_df['log_views_to_likes_ratio'], kde=True, bins=30, color='skyblue')
plt.title('Distribution of Log-Transformed YouTube Views-to-Likes Ratio')
plt.xlabel('Log-Transformed Views-to-Likes Ratio')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

Now the distribution of the ratio appears normal with the highest frequency at a ratio of 5. We should not forget this is a natural logarithm transformation of our original values, so in order to better understand the meaning of this distribution we need to reverse transform the log.

In [ ]:
original_ratio_peak = np.exp(5) - 1
original_ratio_peak

This would mean that, on average, the most common views-to-likes ratio is around 147, which implies that for every like, these songs receive 147 views.

### Problem 8. TikTok stuff (2 points)
The most popular songs on TikTok released every year show... interesting behavior. Which years peaked the most TikTok views? Show an appropriate chart. Can you explain this behavior? For a bit of context, TikTok was created in 2016.

Now, how much popular is the most popular song for each release year, than the mean popularity? Analyze the results.

In both parts, it would be helpful to see the actual songs.

In [ ]:
spotify_df['release_year'] = spotify_df['release_date'].dt.year

yearly_tiktok_views = spotify_df.groupby('release_year')['tiktok_views'].sum()

plt.figure(figsize=(12, 6))
sns.barplot(x=yearly_tiktok_views.index, y=yearly_tiktok_views.values, color='skyblue')
plt.title('Total TikTok Views by Release Year')
plt.xlabel('Release Year')
plt.ylabel('Total TikTok Views')
plt.xticks(rotation=45)
plt.show()


Even though TikTok was created in 2016, the presence of early releases on the platform could be due to several reasons: 

* historical songs gaining popularity because of trends, challenges or virality
* data entry errors
* remixes or re-releases on old songs leading to renewed popularity on TikTok

We can try adjusting the release year filter to see what results this will yield.

In [ ]:
filtered_df = spotify_df[spotify_df['release_year'] >= 2016]

yearly_tiktok_views = filtered_df.groupby('release_year')['tiktok_views'].sum()

most_popular_songs = filtered_df.loc[filtered_df.groupby('release_year')['tiktok_views'].idxmax()]
most_popular_songs_info = most_popular_songs[['release_year', 'track', 'artist', 'tiktok_views']]


plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x=yearly_tiktok_views.index, y=yearly_tiktok_views.values, color='skyblue')
plt.title('Total TikTok Views by Release Year')
plt.xlabel('Release Year')
plt.ylabel('Total TikTok Views')
plt.xticks(rotation=45)

for index, row in most_popular_songs_info.iterrows():
    bar_plot.text(row['release_year'] - 2016, yearly_tiktok_views[row['release_year']], row['track'], 
                  color='black', ha="center", va="bottom")
    
plt.show()

We can see that since the establishment of TikTok in 2016 there had been an upward trend, peaking in 2022, followed by a decline in 2023 and 2024. The low levels of views in 2024 can be easily explained by the incompleteness of the data, as the year has not passed yet, though the low numbers in 2023 are surprising and could indicate several implications:

* peak popularity and oversaturation of the social network
* global trends and challenges, viral in 2022
* algorithm changes might have caused this pattern, as well as user behaviour adjusting to them
* external factors, such as the coronavirus pandemic and competition from other platforms
* data collection regulations could affect the visibility of TikTok views

Now, we will proceed to investigating the popularity of the most popular song in each year, since 2016, against the average song popularity in the given year. In order to do this, we need to first filter songs from 2016 onward and then take the average views for each year. The next step is to identify the top songs on an anual basis and gather their information, which will be required for our plot later. Then we make the comparison by taking the ratio of max views to mean views. From this step onward, the only thing left to do is to plot a bar chart, representing the popularity of the top song to the average popularity of songs that same year.

In [ ]:
filtered_df = spotify_df[spotify_df['release_year'] >= 2016]
mean_views_per_year = filtered_df.groupby('release_year')['tiktok_views'].mean()

most_popular_songs = filtered_df.loc[filtered_df.groupby('release_year')['tiktok_views'].idxmax()]
most_popular_songs_info = most_popular_songs[['release_year', 'track', 'artist', 'tiktok_views']]

popularity_comparison = most_popular_songs_info.set_index('release_year').join(mean_views_per_year, rsuffix='_mean')
popularity_comparison.rename(columns={'tiktok_views': 'max_views', 'tiktok_views_mean': 'mean_views'}, inplace=True)
popularity_comparison['popularity_ratio'] = popularity_comparison['max_views'] / popularity_comparison['mean_views']

plt.figure(figsize=(12, 6))
bar_plot = sns.barplot(x=popularity_comparison.index, y=popularity_comparison['popularity_ratio'], color='coral')
plt.title('Most Popular Song Views to Mean Views Ratio by Release Year')
plt.xlabel('Release Year')
plt.ylabel('Ratio of Most Popular Song Views to Mean Views')
plt.xticks(rotation=45)

for index, row in popularity_comparison.iterrows():
    bar_plot.text(index - 2016, row['popularity_ratio'], row['track'], color='black', ha="center", va="bottom")

plt.show()

This plot indicates that the song 'oh no' in 2019 was significantly more popular than all other songs during that year, whereas, as a comparison, in 2021 the popularity of 'stay' was just slightly higher than average. This plot shows the variations in how much the top song dominated the average song in each year. A high ratio means a song much more popular, while a low ratio means the top song was closer to the average in the given year.

### * Problem 9. Explore (and clean) at will
There is a lot to look for here. For example, you can easily link a song to its genres, and lyrics. You may also try to link artists and albums to more info about them. Or you can compare and contrast a song's performance across different platforms, in a similar manner to what you already did above; maybe even assign a better song ranking system (across platforms with different popularity metrics, and different requirements) than the one provided in the dataset.